# 데이터셋 합치기

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm
import re

import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

## 1. Train, Validation 데이터셋 합치기

### 1-1. 2022년 데이터셋 통합

In [2]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"현재 디렉토리: {original_directory}")

현재 디렉토리: c:\Users\aryij\Documents\DataStudy\epoch\final-project


In [3]:
def combine_datasets(dir1="aihub", dir2="2022"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2)

    # 2022, 2023 폴더 내부 (수도권, 동부권, 서부권, 제주도)를 region_list로 선언
    region_list = os.listdir(base_path)
    region_eng = ["capital", "east", "west", "jeju"]
    
    region_dfs = {}
    region_gps_dfs = {}


    for region, eng_region in zip(region_list, region_eng):

        print(f"{region} 데이터셋 통합 시작")

        # 지역에 따른 경로 설정
        region_path = os.path.join(base_path, region)

        if dir2=="2023":
            # 지역\3.개방데이터로 이동
            mid_path = os.path.join(region_path, os.listdir(region_path)[0]) 

            # 첫 번째 폴더 내부의 첫 번째 하위 폴더로 이동 (지역\3.개방데이터\1.데이터)
            data_path = os.path.join(mid_path, os.listdir(mid_path)[0])
        
        else:
            # 지역\01-1.정식개방데이터로 이동
            data_path = os.path.join(region_path, os.listdir(region_path)[0])  

        # train, validation 데이터셋 combine하기 위해 경로 설정 및 폴더 생성
        combine_path = os.path.join(data_path, "combined")

        # 기존 폴더가 있다면 삭제
        # if os.path.exists(combine_path):
        #     shutil.rmtree(combine_path)

        # os.makedirs(combine_path, exist_ok=True)

        # 01-1.정식개방데이터 내부 폴더 리스트 가져온다
        train_val_list = os.listdir(data_path)

        # data_dict, gps_dict = {}, {}

        region_dfs[eng_region] = {}
        region_gps_dfs[eng_region] = {}

        for folder in train_val_list:
            if (folder == "combined") or (folder == "Other") or (folder == "concatenated"):
                continue
            
            # 02.라벨링데이터 폴더 내부로 이동
            folder_path = os.path.join(data_path, folder, "02.라벨링데이터")

            # .zip (압축파일)이 경로에 있는 경우 리스트에서 뺀다
            labeling_folders = [f for f in os.listdir(folder_path) if not f.endswith(".zip")]

            for subfolder in labeling_folders:
                # 02.라벨링데이터 내부 경로 가져온다
                subfolder_path = os.path.join(folder_path, subfolder)

                # csv_list가 메타데이터라면 불러오지 않는다
                csv_files = [f for f in os.listdir(subfolder_path) if not "poi" in f]

                for csv_file in csv_files:
                    # csv 파일 경로 불러오기
                    file_path = os.path.join(subfolder_path, csv_file)
                    # .csv를 빈 칸으로 replace
                    df_name = csv_file.replace(".csv", "")
                    
                    # 이름 중복 검사 및 수정 (data_dict와 gps_dict 모두 확인)
                    if df_name in region_dfs[eng_region].keys() or df_name in region_gps_dfs.keys():
                        df_name = f"{df_name}_val"  # 중복 시 '이름 뒤에 _val' 추가

                    df = pd.read_csv(file_path)
                    
                    # gps 데이터인 경우 바로 combine_path에 저장
                    if "gps" in df_name.lower():
                        region_gps_dfs[eng_region][df_name] = df

                    else:
                        region_dfs[eng_region][df_name] = df
                        # region_dfs = make_region_column(region_dfs)

        # region_dfs[eng_region] 파일의 key를 순서대로 정렬한 후 list로 변경
        key_list = sorted(region_dfs[eng_region].keys())
        print(f"{region} 데이터: {key_list}\n")
        key_pairs = [(key_list[i], key_list[i + 1]) for i in range(0, len(key_list), 2)]

        for key_train, key_val in key_pairs:
            # 데이터프레임 세로로 병합
            # combined_df = pd.combine([region_dfs[eng_region][key_train], region_dfs[eng_region][key_val]], axis=0)
            all_data = []
            all_data.extend(region_dfs[eng_region][key_train].to_dict(orient="records"))
            all_data.extend(region_dfs[eng_region][key_val].to_dict(orient="records"))

            # combine 결과를 저장
            region_dfs[eng_region][f"{key_train}_combine"] = pd.DataFrame(all_data)
            
            # print(f"{key_train} Train 데이터셋 사이즈:      {region_dfs[eng_region][key_train].shape[0]}")
            # print(f"{key_train} Validation 데이터셋 사이즈: {region_dfs[eng_region][key_val].shape[0]}")
            # print(f"{key_train} Combined 데이터셋 사이즈:   {pd.DataFrame(all_data).shape[0]}")
            # print("------------------------------------------------------------------------------\n")

        print(f"========================== {region} 데이터 통합 완료 ==========================\n")

    return region_dfs, region_gps_dfs

In [4]:
# 2022년 데이터셋 combine
region_dfs_22, region_gps_dfs_22 = combine_datasets("aihub", "2022")

277.국내 여행로그 데이터(수도권) 데이터셋 통합 시작
277.국내 여행로그 데이터(수도권) 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_A', 'tn_activity_consume_his_활동소비내역_A_val', 'tn_activity_his_활동내역_A', 'tn_activity_his_활동내역_A_val', 'tn_adv_consume_his_사전소비내역_A', 'tn_adv_consume_his_사전소비내역_A_val', 'tn_companion_info_동반자정보_A', 'tn_companion_info_동반자정보_A_val', 'tn_lodge_consume_his_숙박소비내역_A', 'tn_lodge_consume_his_숙박소비내역_A_val', 'tn_move_his_이동내역_A', 'tn_move_his_이동내역_A_val', 'tn_mvmn_consume_his_이동수단소비내역_A', 'tn_mvmn_consume_his_이동수단소비내역_A_val', 'tn_tour_photo_관광사진_A', 'tn_tour_photo_관광사진_A_val', 'tn_travel_여행_A', 'tn_travel_여행_A_val', 'tn_traveller_master_여행객 Master_A', 'tn_traveller_master_여행객 Master_A_val', 'tn_visit_area_info_방문지정보_A', 'tn_visit_area_info_방문지정보_A_val']

========================== 277.국내 여행로그 데이터(수도권) 데이터 통합 완료 ==========================

278.국내 여행로그 데이터(동부권) 데이터셋 통합 시작
278.국내 여행로그 데이터(동부권) 데이터: ['tc_c

### 1-2. 2023년 데이터셋 통합

In [5]:
# 2023년 데이터셋 combine
region_dfs_23, region_gps_dfs_23 = combine_datasets("aihub", "2023")

145.국내 여행로그 데이터_수도권_2차년도 데이터셋 통합 시작
145.국내 여행로그 데이터_수도권_2차년도 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_E', 'tn_activity_consume_his_활동소비내역_E_val', 'tn_activity_his_활동내역_E', 'tn_activity_his_활동내역_E_val', 'tn_adv_consume_his_사전소비내역_E', 'tn_adv_consume_his_사전소비내역_E_val', 'tn_companion_info_동반자정보_E', 'tn_companion_info_동반자정보_E_val', 'tn_lodge_consume_his_숙박소비내역_E', 'tn_lodge_consume_his_숙박소비내역_E_val', 'tn_move_his_이동내역_E', 'tn_move_his_이동내역_E_val', 'tn_mvmn_consume_his_이동수단소비내역_E', 'tn_mvmn_consume_his_이동수단소비내역_E_val', 'tn_tour_photo_관광사진_E', 'tn_tour_photo_관광사진_E_val', 'tn_travel_여행_E', 'tn_travel_여행_E_val', 'tn_traveller_master_여행객 Master_E', 'tn_traveller_master_여행객 Master_E_val', 'tn_visit_area_info_방문지정보_E', 'tn_visit_area_info_방문지정보_E_val']

========================== 145.국내 여행로그 데이터_수도권_2차년도 데이터 통합 완료 ==========================

146.국내 여행로그 데이터_동부권_2차년도 데이터셋 통합 시작
146.국내 여행로그 데이터_

In [6]:
# size = 0
# for df in region_gps_dfs_23["capital"].keys():
#     size += region_gps_dfs_23["capital"][df].shape[0]
# size

In [7]:
# 2022년 gps 데이터 capital train + validation 3600개 동일
len(region_gps_dfs_22["capital"].keys())

3600

In [8]:
# 2023년 gps 데이터 capital train + validation 2880개 동일
len(region_gps_dfs_23["capital"].keys())

2880

In [9]:
# 2022년 gps 데이터 jeju train + validation 3600개 동일
len(region_gps_dfs_22["jeju"].keys())

3600

In [10]:
# 2023년 gps 데이터 jeju train + validation 2880개 동일
len(region_gps_dfs_23["jeju"].keys())

2880

In [11]:
len(region_gps_dfs_22["capital"].keys())

3600

In [12]:
region_gps_dfs_22["capital"].keys() == region_gps_dfs_22["east"].keys()

False

In [13]:
# 모든 딕셔너리의 키를 집합으로 변환
key_sets = [set(d.keys()) for d in [region_gps_dfs_22["capital"], 
                                    region_gps_dfs_22["east"], 
                                    region_gps_dfs_22["west"],
                                    region_gps_dfs_22["jeju"]]]

# 모든 딕셔너리 쌍의 키를 비교
for i in range(len(key_sets)):
    for j in range(i + 1, len(key_sets)):
        difference = key_sets[i] ^ key_sets[j]  # 대칭 차집합
        if difference:
            print(f"딕셔너리 {i+1}와 딕셔너리 {j+1}의 키 차이 ({len(difference)}개): {difference}")
        else:
            print(f"딕셔너리 {i+1}와 딕셔너리 {j+1}의 키가 동일합니다.")

# 로그데이터 파일 이름 전부 다르다

딕셔너리 1와 딕셔너리 2의 키 차이 (7200개): {'tn_gps_coord_a_a017674', 'tn_gps_coord_a_a017723', 'tn_gps_coord_b_b006603', 'tn_gps_coord_a_a000715', 'tn_gps_coord_b_b011224', 'tn_gps_coord_b_b003089', 'tn_gps_coord_a_a003818', 'tn_gps_coord_a_a013998', 'tn_gps_coord_a_a007774', 'tn_gps_coord_b_b007894', 'tn_gps_coord_a_a003430', 'tn_gps_coord_b_b011290', 'tn_gps_coord_a_a011117', 'tn_gps_coord_a_a007374', 'tn_gps_coord_a_a017451', 'tn_gps_coord_a_a006028', 'tn_gps_coord_b_b012534', 'tn_gps_coord_b_b000213', 'tn_gps_coord_b_b003303', 'tn_gps_coord_b_b009405', 'tn_gps_coord_b_b018725', 'tn_gps_coord_b_b010105', 'tn_gps_coord_a_a001450', 'tn_gps_coord_b_b007929', 'tn_gps_coord_b_b008004', 'tn_gps_coord_a_a005606', 'tn_gps_coord_b_b009752', 'tn_gps_coord_a_a010570', 'tn_gps_coord_a_a007004', 'tn_gps_coord_a_a006582', 'tn_gps_coord_b_b003186', 'tn_gps_coord_b_b000477', 'tn_gps_coord_a_a000635', 'tn_gps_coord_a_a005206', 'tn_gps_coord_a_a002206', 'tn_gps_coord_b_b006153', 'tn_gps_coord_a_a004606', 'tn_gps

In [14]:
# 중복되는 데이터 확인 → 로그 데이터의 DT_MIN이 달라도 중복데이터로 취급
# 로그데이터는 활동내역, 방문지, 소비내역 등 기록
# 같은 시간에 같은 장소에서 여러번 발생할 수 있다 → 중복 제거하지 않는다
region_gps_dfs_22["capital"]["tn_gps_coord_a_a002571"][region_gps_dfs_22["capital"]["tn_gps_coord_a_a002571"].duplicated()]

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID
1,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:16,a_a002571
3,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:17,a_a002571
5,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:18,a_a002571
9,96ffd6494524add7,126.711467,37.460012,2022-10-02 08:21,a_a002571
10,96ffd6494524add7,126.711467,37.460012,2022-10-02 08:21,a_a002571
...,...,...,...,...,...
1351,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:00,a_a002571
1352,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:00,a_a002571
1354,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:01,a_a002571
1356,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:02,a_a002571


## 2. 파생 컬럼 생성

### 2-1. REGION 컬럼 생성

In [6]:
# 통합한 데이터셋에 region 컬럼 생성하는 함수
def make_region_column(region_dict):

    # 딕셔너리 key를 받아온다
    region_keys = region_dict.keys()
    
    # region_keys: capital, east, west, jeju
    for region_key in region_keys:
        
        # region_key를 바탕으로, 각각의 지역 필터링한 후 keys를 다시 찾는다 (데이터셋 이름들)
        dataset_keys = region_dict[region_key].keys()

        for dataset_key in dataset_keys:
            # 지역별 데이터셋들을 각각 불러와서 region 컬럼을 생성하고 해당 지역을 값에 넣는다
            regional_data = region_dict[region_key][dataset_key]
            regional_data["REGION"] = f"{region_key}"
    
    return region_dict

In [7]:
# 22년, 23년 데이터에 make_region_column 함수 적용
region_combined_dfs_22 = make_region_column(region_dfs_22)
region_combined_gps_dfs_22 = make_region_column(region_gps_dfs_22)

region_combined_dfs_23 = make_region_column(region_dfs_23)
region_combined_gps_dfs_23 = make_region_column(region_gps_dfs_23)

In [17]:
# region_combined_gps_dfs_22 확인
region_combined_gps_dfs_22["capital"]["tn_gps_coord_a_a000011"].head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID,REGION
0,a9c1a065c89da3b7,126.702718,37.500779,2022-08-06 12:33,a_a000011,capital
1,a9c1a065c89da3b7,126.703540,37.506400,2022-08-06 12:34,a_a000011,capital
2,a9c1a065c89da3b7,126.706564,37.506687,2022-08-06 12:35,a_a000011,capital


In [18]:
# region_combined_gps_dfs_23 확인
region_combined_gps_dfs_23["east"]["tn_gps_coord_e_e000003"].head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID,REGION
0,e000003_900008,126.702967,37.628667,2023-04-30 07:19,e_e000003,east
1,e000003_900008,126.702967,37.628667,2023-04-30 07:20,e_e000003,east
2,e000003_900008,126.702967,37.628667,2023-04-30 07:21,e_e000003,east


In [8]:
# 2022년 데이터셋 train, validation combine 제대로 되었는지 확인

print("tn_activity_consume_his_활동소비내역_D Train 데이터:     ", region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D"].shape)
print("tn_activity_consume_his_활동소비내역_D Validation 데이터: ", region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_val"].shape)
print("tn_activity_consume_his_활동소비내역_D Combine 데이터:    ", region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_combine"].shape)

if region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D"].shape[0] + region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_val"].shape[0] == region_combined_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_combine"].shape[0]:
    print("\n데이터 combine이 제대로 되었습니다!")
else:
    print("\n데이터 combine에 문제가 생겼습니다.")

tn_activity_consume_his_활동소비내역_D Train 데이터:      (33796, 19)
tn_activity_consume_his_활동소비내역_D Validation 데이터:  (4224, 19)
tn_activity_consume_his_활동소비내역_D Combine 데이터:     (38020, 19)

데이터 combine이 제대로 되었습니다!


In [20]:
# 2023년 데이터셋 train, validation combine 제대로 되었는지 확인

print("tn_activity_consume_his_활동소비내역_H Train 데이터:     ", region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H"].shape)
print("tn_activity_consume_his_활동소비내역_H Validation 데이터: ", region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_val"].shape)
print("tn_activity_consume_his_활동소비내역_H Combine 데이터:    ", region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_combine"].shape)

if region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H"].shape[0] + region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_val"].shape[0] == region_combined_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_combine"].shape[0]:
    print("\n데이터 combine이 제대로 되었습니다!")
else:
    print("\n데이터 combine에 문제가 생겼습니다.")

tn_activity_consume_his_활동소비내역_H Train 데이터:      (26942, 19)
tn_activity_consume_his_활동소비내역_H Validation 데이터:  (3184, 19)
tn_activity_consume_his_활동소비내역_H Combine 데이터:     (30126, 19)

데이터 combine이 제대로 되었습니다!


### 2-2. 날짜, 시간 컬럼 분리

In [9]:
# 컬럼 이름 DT or DT_MIN으로 끝나는 컬럼 찾아서 날짜, 시간 분리
# 특정 키워드(IN, OUT, START, END)가 포함된 경우 데이터프레임 내 두 개의 컬럼 모두 변경

def split_datetime_columns(region_dict):

    # 딕셔너리 key를 받아온다
    region_keys = region_dict.keys()
    
    for region_key in region_keys:
        dataset_keys = region_dict[region_key].keys()
        
        for dataset_key in dataset_keys:
            df = region_dict[region_key][dataset_key]

            # 데이터프레임에서 "DT" 또는 "DT_MIN"으로 끝나는 컬럼 찾는다
            datetime_columns = [col for col in df.columns if col.endswith('DT') or col.endswith('DT_MIN')]
            
            for col in datetime_columns:
                # datetime 형식 변환
                original_nan_count = df[col].isna().sum()
                df[col] = pd.to_datetime(df[col], errors='coerce')
                
                # NaT 갯수 확인
                nat_count = df[col].isna().sum()
                if nat_count > 0:
                    print(f"datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):\n- 데이터프레임: {dataset_key}\n- 컬럼: {col}\n- NaT 갯수: {nat_count}")
                    print(f"- NaN 갯수: {original_nan_count} (원본 데이터셋 동일 컬럼)\n")

                # 키워드에 따라 prefix 설정
                prefix = None
                if "START" in col:
                    prefix = "START"
                elif "END" in col:
                    prefix = "END"
                elif "IN" in col and "MIN" not in col:  # "IN" 포함, "MIN" 제외
                    prefix = "CHK_IN"
                elif "OUT" in col:
                    prefix = "CHK_OUT"

                # 새로운 컬럼 이름 생성
                date_col = f"{prefix}_DT_YMD" if prefix else f"{col}_YMD"
                time_col = f"{prefix}_DT_MIN" if prefix else f"{col}_MIN"

                if time_col.endswith('MIN_MIN'):
                    time_col = time_col[:-4]

                if date_col.endswith('MIN_YMD'):
                    date_col = date_col.split("_")[0] + "_" + date_col.split("_")[-1]


                    
                # 새로운 컬럼 추가 (날짜와 시간)
                df[date_col] = df[col].dt.date
                df[time_col] = df[col].dt.time


                # 컬럼 순서 조정: DATE를 TIME 앞에 위치
                cols = df.columns.tolist()
                date_idx = cols.index(date_col)
                time_idx = cols.index(time_col)
                
                # 순서 변경: DATE -> TIME
                cols.insert(date_idx + 1, cols.pop(time_idx))  # DATE를 TIME 앞에 위치
                df = df[cols]  # 새로운 순서로 데이터프레임 정렬
            
            # 처리된 데이터프레임 업데이트
            region_dict[region_key][dataset_key] = df
            
        print(f"========================== {region_key} 지역 날짜 컬럼 분리 완료 ==========================\n\n")
    
    return region_dict


In [10]:
# region_dfs_23_copy = copy.deepcopy(region_dfs_23)
# region_gps_dfs_23_copy = copy.deepcopy(region_gps_dfs_23)

In [11]:
date_separated_dfs_22 = split_datetime_columns(region_combined_dfs_22)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_activity_consume_his_활동소비내역_A
- 컬럼: PAYMENT_DT
- NaT 갯수: 2405
- NaN 갯수: 2405 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_adv_consume_his_사전소비내역_A
- 컬럼: PAYMENT_DT
- NaT 갯수: 298
- NaN 갯수: 298 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_A
- 컬럼: CHK_IN_DT_MIN
- NaT 갯수: 49
- NaN 갯수: 49 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_A
- 컬럼: CHK_OUT_DT_MIN
- NaT 갯수: 49
- NaN 갯수: 49 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_A
- 컬럼: PAYMENT_DT
- NaT 갯수: 613
- NaN 갯수: 613 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_move_his_이동내역_A
- 컬럼: START_DT_MIN
- NaT 갯수: 26953
- NaN 갯수: 26953 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_move_his_이동내역_A
- 컬럼: END_DT_MIN
- NaT 갯수: 3192
- NaN 갯수: 3192 (원본 데이터셋 동일 컬럼)



In [12]:
date_separated_dfs_23 = split_datetime_columns(region_combined_dfs_23)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_activity_consume_his_활동소비내역_E
- 컬럼: PAYMENT_DT
- NaT 갯수: 1378
- NaN 갯수: 1378 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_adv_consume_his_사전소비내역_E
- 컬럼: PAYMENT_DT
- NaT 갯수: 193
- NaN 갯수: 193 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_E
- 컬럼: CHK_IN_DT_MIN
- NaT 갯수: 745
- NaN 갯수: 745 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_E
- 컬럼: CHK_OUT_DT_MIN
- NaT 갯수: 745
- NaN 갯수: 745 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_lodge_consume_his_숙박소비내역_E
- 컬럼: PAYMENT_DT
- NaT 갯수: 284
- NaN 갯수: 284 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_move_his_이동내역_E
- 컬럼: START_DT_MIN
- NaT 갯수: 18821
- NaN 갯수: 18821 (원본 데이터셋 동일 컬럼)

datetime 수정 과정에서 NaT 발생 (%Y-%m-%d %H:%M):
- 데이터프레임: tn_move_his_이동내역_E
- 컬럼: END_DT_MIN
- NaT 갯수: 2563
- NaN 갯수: 2563 (원본 데이터셋 동일 컬

In [57]:
date_separated_gps_dfs_22 = split_datetime_columns(region_combined_gps_dfs_22)

========================== capital 지역 날짜 컬럼 분리 완료 ==========================


========================== east 지역 날짜 컬럼 분리 완료 ==========================


========================== west 지역 날짜 컬럼 분리 완료 ==========================


========================== jeju 지역 날짜 컬럼 분리 완료 ==========================




In [58]:
date_separated_gps_dfs_23 = split_datetime_columns(region_combined_gps_dfs_23)

========================== capital 지역 날짜 컬럼 분리 완료 ==========================


========================== east 지역 날짜 컬럼 분리 완료 ==========================


========================== west 지역 날짜 컬럼 분리 완료 ==========================


========================== jeju 지역 날짜 컬럼 분리 완료 ==========================




In [13]:
# 시간 나뉘어진 것 확인
date_separated_dfs_22["capital"]["tn_activity_consume_his_활동소비내역_A_combine"].head()

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REGION,PAYMENT_DT_YMD,PAYMENT_DT_MIN
0,a_a004739,2210020002,3,0,0,1,2,NaN,아침고요수목원,경기 가평군 상면 수목원로 432,경기 가평군 상면 행현리 623-3,3216056.0,4.182033e+09,2022-10-02 00:00:00,1.0,22000.0,대인,NaN,capital,2022-10-02,00:00:00
1,a_a002397,2209170002,3,1,1,0,1,NaN,에버랜드,에버랜드로 199 삼성물산(주),NaN,NaN,NaN,NaT,4.0,33000.0,NaN,4.146125e+09,capital,NaT,NaT
2,a_a017904,2211110007,1,0,0,1,2,5.918702e+09,금별맥주,서울 마포구 마포대로 110,서울 마포구 공덕동 255-16,2113001.0,1.144010e+09,2022-11-11 22:32:00,1.0,31400.0,봄베이토닉하이볼;호세쿠엘보;바지락버터술찜;파스타면,NaN,capital,2022-11-11,22:32:00
3,a_a018111,2211130004,1,0,0,1,2,3.328101e+09,주식회사필모어,경기 고양시 일산동구 일산로380번길 15,경기 고양시 일산동구 정발산동 1296-6,4379164.0,4.128510e+09,2022-11-13 14:10:00,1.0,48000.0,4.LASAGNA;7.GNOCCHI;COKE,NaN,capital,2022-11-13,14:10:00
4,a_a005012,2210120010,1,0,0,1,1,6.448101e+09,박가네푸드시스템 주식회사,서울 종로구 종로32길 7,서울 종로구 종로5가 138-10,4100322.0,1.111016e+09,2022-10-12 16:26:00,1.0,10000.0,NaN,NaN,capital,2022-10-12,16:26:00


In [60]:
date_separated_dfs_22["jeju"]["tn_move_his_이동내역_D_combine"].head(10)

,TRAVEL_ID,TRIP_ID,START_VISIT_AREA_ID,END_VISIT_AREA_ID,MVMN_CD_1,MVMN_CD_2,REGION,START_DT_YMD,START_DT_MIN,END_DT_YMD,END_DT_MIN
0,d_d005326,2210100002,NaN,2.210100e+09,2.0,13.0,jeju,NaT,NaT,2022-10-10,20:30:00
1,d_d010442,2211030001,NaN,2.211030e+09,2.0,NaN,jeju,NaT,NaT,2022-11-03,20:30:00
2,d_d001212,2209080004,NaN,2.209080e+09,16.0,NaN,jeju,NaT,NaT,2022-09-08,14:30:00
3,d_d010496,2211040005,NaN,2.211040e+09,1.0,NaN,jeju,NaT,NaT,2022-11-04,15:00:00
4,d_d002566,2210010010,NaN,2.210010e+09,15.0,NaN,jeju,NaT,NaT,2022-10-01,20:00:00
5,d_d002711,2210230007,NaN,2.210230e+09,15.0,NaN,jeju,NaT,NaT,2022-10-23,21:00:00
6,d_d004407,2210160001,2.210160e+09,NaN,NaN,NaN,jeju,2022-10-16,10:30:00,NaT,NaT
7,a_a000490,2208200002,NaN,2.208200e+09,1.0,NaN,jeju,NaT,NaT,2022-08-20,10:00:00
8,b_b002600,2208200003,NaN,2.208200e+09,10.0,15.0,jeju,NaT,NaT,2022-08-20,12:00:00
9,d_d001896,2210080006,NaN,2.210080e+09,2.0,NaN,jeju,NaT,NaT,2022-10-08,17:30:00


In [61]:
date_separated_gps_dfs_22["capital"]["tn_gps_coord_a_a000011"].head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,a9c1a065c89da3b7,126.702718,37.500779,a_a000011,capital,2022-08-06,12:33:00
1,a9c1a065c89da3b7,126.703540,37.506400,a_a000011,capital,2022-08-06,12:34:00
2,a9c1a065c89da3b7,126.706564,37.506687,a_a000011,capital,2022-08-06,12:35:00


### 2-3. 여행 시작, 끝 날짜 컬럼 생성
- tn_traveller_master_여행객 Master.csv 파일의 `TRAVEL_STATUS_YMD`
    - 2023-07-16~2023-07-16의 형태로 되어있다
- 해당 컬럼의 데이터를 시작 날짜, 도착 날짜 컬럼으로 나눈다

In [34]:
date_separated_dfs_22["capital"]["tn_traveller_master_여행객 Master_A_combine"].head(3)

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,REGION
0,b015583,41,여,20,6,1.0,1.0,5,3.0,NaN,3,7.0,2,2,50,50110,26,26500,47,47130,1,1,1,4,7,1,1,7,경기도,충남,2인 여행(가족 외),2022-10-29~2022-10-30,1,NaN,NaN,1,capital
1,a001105,30,남,30,6,1.0,1.0,2,3.0,NaN,4,6.0,3,1,41,41460,11,11560,26,26710,6,2,2,2,4,6,6,5,대전광역시,경기,3인 이상 여행(가족 외),2022-09-03~2022-09-04,3,NaN,NaN,7,capital
2,a001673,41,여,30,4,1.0,2.0,3,NaN,1.0,5,8.0,2,3,50,50130,28,28110,26,26710,4,1,2,1,2,5,2,7,경기도,서울,자녀 동반 여행,2022-09-18~2022-09-19,10,7.0,1.0,2,capital


In [36]:
date_separated_dfs_23["capital"]["tn_traveller_master_여행객 Master_E_combine"].head(3)

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,REGION
0,e004720,41,여,60,4,1.0,3,3,11,NaN,4,9.0,2,2,11,11500,47,47760,41,41390,2,4,4,4,4,4,5,5,경기도,서울,2인 가족 여행,2023-07-16~2023-07-16,2,6.0,NaN,1,capital
1,e000914,30,여,20,6,1.0,1,1,3,NaN,4,NaN,3,2,45,45110,26,26230,11,11650,3,6,2,3,2,2,4,1,대전광역시,서울,나홀로 여행,2023-06-03~2023-06-03,1,7.0,10.0,0,capital
2,e003564,41,여,30,7,1.0,2,4,2,NaN,7,12.0,2,2,42,42210,48,48840,41,41480,1,1,1,3,2,1,1,7,경기도,경기,자녀 동반 여행,2023-06-24~2023-06-24,8,3.0,7.0,3,capital


In [14]:
def seperate_ymd(data_dict, col_name_1, col_name_2):

    keys_region = data_dict.keys()

    for region in keys_region:
        master = "master"
        master_keys = [key for key in data_dict[region].keys() if master in key]
        
        if not master_keys:  # master_key가 비어있다면 건너뜁니다.
            continue

        for master_key in master_keys:
            df = data_dict[region][master_key]

            df[col_name_1] = df["TRAVEL_STATUS_YMD"].str.split("~").str[0]
            df[col_name_2] = df["TRAVEL_STATUS_YMD"].str.split("~").str[1]

            data_dict[region][master_key] = df

    return data_dict

In [15]:
# aa = copy.deepcopy(date_separated_dfs_22)

In [16]:
date_separated_dfs_22 = seperate_ymd(date_separated_dfs_22, "TRAVEL_STATUS_START_YMD", "TRAVEL_STATUS_END_YMD")
date_separated_dfs_23 = seperate_ymd(date_separated_dfs_23, "TRAVEL_STATUS_START_YMD", "TRAVEL_STATUS_END_YMD")

## 3. 데이터프레임 이름 변경
- train, validation 데이터를 combine한 데이터프레임의 이름이 `column_combine` 형식으로 되어있다
- `_combine` 글자를 제거한다

In [17]:
# train, validation combine한 데이터프레임 이름 변경 함수 (_combine 글자 제거)
def update_keys(data_dict, new_keys):

    # data_dict 딕셔너리의 딥 카피 생성 (원본은 보존하기 위함)
    data_dict_copy = copy.deepcopy(data_dict)
    
    for region, datasets in data_dict_copy.items():
        # "combine" 포함된 key만 필터링
        keys_to_update = [key for key in datasets.keys() if "combine" in key]
        
        # 새 키로 변경
        for idx, key in enumerate(keys_to_update):
            if idx < len(new_keys):  # new_keys 범위 내에서 변경
                new_key = new_keys[idx]
                datasets[new_key] = datasets.pop(key)
                # print(f"{region} {datasets} 크기:      {datasets[new_key].shape[0]}".rjust(5))

        # "combine"로 변경된 key만 남기도록 필터링
        keys_to_remove = [key for key in datasets.keys() if key not in new_keys]
        for key in keys_to_remove:
            datasets.pop(key)
    
    return data_dict_copy

In [18]:
# _combine으로 되어있는 key 값들에서 combine 제외
keys_combine = ['tc_codea_코드A',
               'tc_codeb_코드B',
               'tc_sgg_시군구코드',
               'tn_activity_consume_his_활동소비내역',
               'tn_activity_his_활동내역',
               'tn_adv_consume_his_사전소비내역',
               'tn_companion_info_동반자정보',
               'tn_lodge_consume_his_숙박소비내역',
               'tn_move_his_이동내역',
               'tn_mvmn_consume_his_이동수단소비내역',
               'tn_tour_photo_관광사진',
               'tn_travel_여행',
               'tn_traveller_master_여행객 Master',
               'tn_visit_area_info_방문지정보']

date_separated_dfs_22_updated = update_keys(date_separated_dfs_22, keys_combine)

In [19]:
date_separated_dfs_23_updated = update_keys(date_separated_dfs_23, keys_combine)

In [20]:
# 필터링되고, 바뀐 key 값들 확인
for region, types in date_separated_dfs_22_updated.items():
    print(f"updated_dfs_22 {region}: {list(types.keys())}")

print("")

# 필터링되고, 바뀐 key 값들 확인
for region, types in date_separated_dfs_23_updated.items():
    print(f"updated_dfs_23 {region}: {list(types.keys())}")

updated_dfs_22 capital: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보']
updated_dfs_22 east: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보']
updated_dfs_22 west: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내

In [21]:
# 데이터 숫자 누락 확인 → 누락 없음
print(f'2022년 수도권 tn_move_his_이동내역_A_combine: {region_dfs_22["capital"]["tn_move_his_이동내역_A_combine"].shape[0]}')
print(f'2022년 동부권 tn_activity_consume_his_활동소비내역_B_combine: {region_dfs_22["east"]["tn_activity_consume_his_활동소비내역_B_combine"].shape[0]}')
print('')

print(f'컬럼명 변경한 2022년 수도권 tn_move_his_이동내역: {date_separated_dfs_22_updated["capital"]["tn_move_his_이동내역"].shape[0]}')
print(f'컬럼명 변경한 2022년 동부권 tn_activity_consume_his_활동소비내역: {date_separated_dfs_22_updated["east"]["tn_activity_consume_his_활동소비내역"].shape[0]}')

2022년 수도권 tn_move_his_이동내역_A_combine: 33913
2022년 동부권 tn_activity_consume_his_활동소비내역_B_combine: 22302

컬럼명 변경한 2022년 수도권 tn_move_his_이동내역: 33913
컬럼명 변경한 2022년 동부권 tn_activity_consume_his_활동소비내역: 22302


## 4. Region 통합
- 현재 `data_dict`은 `data_dict["region"]["csv_name"]`의 구조로 되어있다 (Nested Dictionary)
- `region`을 통합해준다

In [22]:
def combine_dict_total_region(data_dict, is_gps="no"):
    combine_total_region = {}

    # 이중 딕셔너리 순회
    for region, datasets in tqdm(data_dict.items()):
        for df_name, df in datasets.items():
            if df_name not in combine_total_region:
                combine_total_region[df_name] = []  # type별로 리스트 초기화
            
            # 각 데이터프레임을 딕셔너리 레코드로 변환 후 리스트에 추가
            combine_total_region[df_name].extend(df.to_dict(orient='records'))

    if is_gps.lower() == "no":
        # 중복 제거 후 각 key에 대해 DataFrame으로 변환
        combine_total_region = {
            key: pd.DataFrame(records).drop_duplicates() for key, records in combine_total_region.items()
        }
    else:
        # gps데이터일 경우, 모든 데이터를 하나의 DataFrame으로 병합
        all_records = []
        for records in combine_total_region.values():
            all_records.extend(records)  # 모든 records를 합침
        combine_total_region = pd.DataFrame(all_records)  # 하나의 DataFrame으로 변환

    return combine_total_region


In [59]:
# updated_dfs_22 -> capital -> 시군구코드
# updated_dfs_22 -> 시군구코드

In [60]:
def divide_dictionary(dictionary, length):
    """
    이중 딕셔너리를 length로 나눕니다.
    length가 각 key의 value 수보다 클 경우, 내부 values를 1/2씩 분배합니다.
    """
    divided_dicts = [{} for _ in range(length)]  # 결과 리스트 초기화

    for key, values in dictionary.items():
        # values가 리스트나 딕셔너리인 경우 처리
        if isinstance(values, (list, dict)):
            items = list(values.items() if isinstance(values, dict) else enumerate(values))
            total_length = len(items)
            divide_length = max(1, total_length // length)  # 각 그룹의 기본 길이
            
            for i in range(length):
                if i == length - 1:  # 마지막 그룹은 나머지 포함
                    divided_dicts[i][key] = dict(items[i * divide_length:])
                else:
                    divided_dicts[i][key] = dict(items[i * divide_length:(i + 1) * divide_length])
        else:
            raise ValueError("dictionary의 value는 리스트나 딕셔너리여야 합니다.")
    
    return divided_dicts

In [71]:
# 2022 데이터 4개로 나눈다
date_separated_gps_dfs_22_1, date_separated_gps_dfs_22_2, date_separated_gps_dfs_22_3, date_separated_gps_dfs_22_4 = divide_dictionary(date_separated_gps_dfs_22, 4)

# 2023 데이터 6개로 나눈다
date_separated_gps_dfs_23_1, date_separated_gps_dfs_23_2, date_separated_gps_dfs_23_3, date_separated_gps_dfs_23_4, date_separated_gps_dfs_23_5, date_separated_gps_dfs_23_6 = divide_dictionary(date_separated_gps_dfs_23, 6)

In [72]:
date_separated_gps_dfs_22_1["capital"].keys()

dict_keys(['tn_gps_coord_a_a000011', 'tn_gps_coord_a_a000012', 'tn_gps_coord_a_a000013', 'tn_gps_coord_a_a000016', 'tn_gps_coord_a_a000018', 'tn_gps_coord_a_a000019', 'tn_gps_coord_a_a000021', 'tn_gps_coord_a_a000028', 'tn_gps_coord_a_a000029', 'tn_gps_coord_a_a000030', 'tn_gps_coord_a_a000034', 'tn_gps_coord_a_a000038', 'tn_gps_coord_a_a000050', 'tn_gps_coord_a_a000056', 'tn_gps_coord_a_a000058', 'tn_gps_coord_a_a000061', 'tn_gps_coord_a_a000065', 'tn_gps_coord_a_a000070', 'tn_gps_coord_a_a000074', 'tn_gps_coord_a_a000080', 'tn_gps_coord_a_a000081', 'tn_gps_coord_a_a000082', 'tn_gps_coord_a_a000084', 'tn_gps_coord_a_a000085', 'tn_gps_coord_a_a000086', 'tn_gps_coord_a_a000087', 'tn_gps_coord_a_a000094', 'tn_gps_coord_a_a000097', 'tn_gps_coord_a_a000100', 'tn_gps_coord_a_a000108', 'tn_gps_coord_a_a000110', 'tn_gps_coord_a_a000111', 'tn_gps_coord_a_a000112', 'tn_gps_coord_a_a000116', 'tn_gps_coord_a_a000117', 'tn_gps_coord_a_a000118', 'tn_gps_coord_a_a000122', 'tn_gps_coord_a_a000123', '

In [73]:
date_separated_gps_dfs_22_2["capital"].keys()

dict_keys(['tn_gps_coord_a_a003033', 'tn_gps_coord_a_a003034', 'tn_gps_coord_a_a003046', 'tn_gps_coord_a_a003049', 'tn_gps_coord_a_a003056', 'tn_gps_coord_a_a003061', 'tn_gps_coord_a_a003062', 'tn_gps_coord_a_a003064', 'tn_gps_coord_a_a003065', 'tn_gps_coord_a_a003066', 'tn_gps_coord_a_a003068', 'tn_gps_coord_a_a003069', 'tn_gps_coord_a_a003071', 'tn_gps_coord_a_a003072', 'tn_gps_coord_a_a003073', 'tn_gps_coord_a_a003074', 'tn_gps_coord_a_a003075', 'tn_gps_coord_a_a003077', 'tn_gps_coord_a_a003078', 'tn_gps_coord_a_a003082', 'tn_gps_coord_a_a003087', 'tn_gps_coord_a_a003089', 'tn_gps_coord_a_a003090', 'tn_gps_coord_a_a003092', 'tn_gps_coord_a_a003093', 'tn_gps_coord_a_a003094', 'tn_gps_coord_a_a003098', 'tn_gps_coord_a_a003104', 'tn_gps_coord_a_a003108', 'tn_gps_coord_a_a003109', 'tn_gps_coord_a_a003110', 'tn_gps_coord_a_a003114', 'tn_gps_coord_a_a003117', 'tn_gps_coord_a_a003121', 'tn_gps_coord_a_a003123', 'tn_gps_coord_a_a003124', 'tn_gps_coord_a_a003125', 'tn_gps_coord_a_a003127', '

In [23]:
# 지역 단위로 나뉘어져 있던 train, validation combine한 데이터프레임을 통합해준다
total_dfs_22 = combine_dict_total_region(date_separated_dfs_22_updated)

100%|██████████| 4/4 [00:14<00:00,  3.71s/it]


In [24]:
total_dfs_23 = combine_dict_total_region(date_separated_dfs_23_updated)

100%|██████████| 4/4 [00:05<00:00,  1.26s/it]


In [76]:
total_gps_dfs_22_1 = combine_dict_total_region(date_separated_gps_dfs_22_1, is_gps="yes")

100%|██████████| 4/4 [00:34<00:00,  8.54s/it]


In [77]:
total_gps_dfs_22_2 = combine_dict_total_region(date_separated_gps_dfs_22_2, is_gps="yes")

100%|██████████| 4/4 [01:05<00:00, 16.38s/it]


In [78]:
total_gps_dfs_22_3 = combine_dict_total_region(date_separated_gps_dfs_22_3, is_gps="yes")

100%|██████████| 4/4 [01:40<00:00, 25.02s/it]


In [79]:
total_gps_dfs_22_4 = combine_dict_total_region(date_separated_gps_dfs_22_4, is_gps="yes")

100%|██████████| 4/4 [01:22<00:00, 20.56s/it]


In [80]:
# total_gps_dfs_23 = combine_dict_total_region(region_gps_dfs_23, is_gps="yes")
total_gps_dfs_23_1 = combine_dict_total_region(date_separated_gps_dfs_23_1, is_gps="yes")

100%|██████████| 4/4 [01:13<00:00, 18.40s/it]


In [81]:
total_gps_dfs_23_2 = combine_dict_total_region(date_separated_gps_dfs_23_2, is_gps="yes")

100%|██████████| 4/4 [01:11<00:00, 17.79s/it]


In [82]:
total_gps_dfs_23_3 = combine_dict_total_region(date_separated_gps_dfs_23_3, is_gps="yes")

100%|██████████| 4/4 [01:29<00:00, 22.28s/it]


In [83]:
total_gps_dfs_23_4 = combine_dict_total_region(date_separated_gps_dfs_23_4, is_gps="yes")

100%|██████████| 4/4 [01:31<00:00, 22.81s/it]


In [84]:
total_gps_dfs_23_5 = combine_dict_total_region(date_separated_gps_dfs_23_5, is_gps="yes")

100%|██████████| 4/4 [01:15<00:00, 18.90s/it]


In [85]:
total_gps_dfs_23_6 = combine_dict_total_region(date_separated_gps_dfs_23_6, is_gps="yes")

100%|██████████| 4/4 [01:07<00:00, 16.76s/it]


In [87]:
print(total_gps_dfs_22_1[total_gps_dfs_22_1["REGION"]=="capital"]["TRAVEL_ID"].nunique())
print(total_gps_dfs_22_1[total_gps_dfs_22_1["REGION"]=="east"]["TRAVEL_ID"].nunique())
print(total_gps_dfs_22_1[total_gps_dfs_22_1["REGION"]=="west"]["TRAVEL_ID"].nunique())
print(total_gps_dfs_22_1[total_gps_dfs_22_1["REGION"]=="jeju"]["TRAVEL_ID"].nunique())

900
900
900
900


In [ ]:
# 예시 데이터 (2023년)
# region 컬럼에 capital 들어간 것 확인
total_gps_dfs_22_4.isnull().sum()

MOBILE_NUM_ID    0
X_COORD          0
Y_COORD          0
TRAVEL_ID        0
region           0
DT_YMD           0
DT_MIN           0
dtype: int64

## 5. 최종적으로 통합한 데이터 저장

In [95]:
# gps 데이터 parquet 파일로 저장
gps_df_dict = {"total_gps_dfs_22_1": total_gps_dfs_22_1, 
               "total_gps_dfs_22_2": total_gps_dfs_22_2, 
               "total_gps_dfs_22_3": total_gps_dfs_22_3, 
               "total_gps_dfs_22_4": total_gps_dfs_22_4,
               "total_gps_dfs_23_1": total_gps_dfs_23_1, 
               "total_gps_dfs_23_2": total_gps_dfs_23_2, 
               "total_gps_dfs_23_3": total_gps_dfs_23_3,
               "total_gps_dfs_23_4": total_gps_dfs_23_4, 
               "total_gps_dfs_23_5": total_gps_dfs_23_5, 
               "total_gps_dfs_23_6": total_gps_dfs_23_6
               }

# 데이터프레임 이름과 함께 파일 저장
for file_name, gps_df in gps_df_dict.items():
    # NaN 값을 대체한 후 문자열로 변환
    gps_df["MOBILE_NUM_ID"] = gps_df["MOBILE_NUM_ID"].fillna("MISSING").astype(str)
    
    if "22" in file_name:
        parquet_path = os.path.join("data", "aihub", "2022", "total_combined", "gps_data_parquet")
        # Parquet 파일 경로
    else:
        parquet_path = os.path.join("data", "aihub", "2023", "total_combined", "gps_data_parquet")
    
    # 디렉토리 생성 
    os.makedirs(parquet_path, exist_ok=True)
    file_path = os.path.join(parquet_path, f"{file_name}.parquet")
    
    # Parquet 파일로 저장
    gps_df.to_parquet(file_path, engine="pyarrow", index=False)
    print(f"Saved {file_path}")


Saved data\aihub\2022\total_combined\gps_data_parquet\total_gps_dfs_22_1.parquet
Saved data\aihub\2022\total_combined\gps_data_parquet\total_gps_dfs_22_2.parquet
Saved data\aihub\2022\total_combined\gps_data_parquet\total_gps_dfs_22_3.parquet
Saved data\aihub\2022\total_combined\gps_data_parquet\total_gps_dfs_22_4.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_1.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_2.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_3.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_4.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_5.parquet
Saved data\aihub\2023\total_combined\gps_data_parquet\total_gps_dfs_23_6.parquet


### 5-1. data_dict (gps 데이터, csv 데이터) 저장

In [ ]:
parquet_path_23 = os.path.join("data", "aihub", "2023", "total_combined", "gps_data_parquet")
pd.read_parquet(os.path.join(parquet_path_23, "total_gps_dfs_23_1.parquet"), engine="pyarrow")

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,e000004_900009,127.054289,37.209919,e_e000004,capital,2023-04-30,13:30:00
1,e000004_900009,127.053911,37.208667,e_e000004,capital,2023-04-30,13:30:00
2,e000004_900009,127.053411,37.207996,e_e000004,capital,2023-04-30,13:31:00
3,e000004_900009,127.052819,37.208729,e_e000004,capital,2023-04-30,13:31:00
4,e000004_900009,127.054071,37.211189,e_e000004,capital,2023-04-30,13:32:00
...,...,...,...,...,...,...,...
17495520,f002706_61611,128.619135,35.856597,f_f002706,jeju,2023-06-04,23:54:00
17495521,f002706_61611,128.619135,35.856597,f_f002706,jeju,2023-06-04,23:55:00
17495522,f002706_61611,128.619135,35.856597,f_f002706,jeju,2023-06-04,23:55:00
17495523,f002706_61611,128.619135,35.856597,f_f002706,jeju,2023-06-04,23:57:00


In [25]:
total_dfs_22.keys()

dict_keys(['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보'])

In [26]:
total_dfs_23.keys()

dict_keys(['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보'])

In [30]:
# parquet 파일로 변환하고 싶은데, 방문지정보.csv 파일의 컬럼 문제로 변환 불가능
# 해당 문제 수정하는 함수수
def change_datatype_of_column(data_dict):
    for dataset in data_dict.keys():
        # 파일 이름에 "방문지정보"가 포함된 경우 추가 처리 (추후 parquet으로 변환 목적)
        if "방문지정보" in dataset:
            df = data_dict[dataset]
            print(f"특정 파일 처리: {dataset}")
            for col in df.columns:
                # 숫자와 문자열이 혼합된 컬럼도 포함하여 처리
                if pd.api.types.is_numeric_dtype(df[col]) or df[col].dtype == 'object':
                    # 숫자형 데이터는 문자열로 변환, NaN과 기존 문자열은 유지
                    df[col] = df[col].apply(
                        lambda x: str(x) if pd.to_numeric(x, errors='coerce') is not None else x
                    )
                    print(f"- 컬럼 {col} 변환 완료")

In [ ]:
change_datatype_of_column(total_dfs_22)

특정 파일 처리: tn_visit_area_info_방문지정보
- 컬럼 VISIT_AREA_ID 변환 완료
- 컬럼 TRAVEL_ID 변환 완료
- 컬럼 VISIT_ORDER 변환 완료
- 컬럼 VISIT_AREA_NM 변환 완료
- 컬럼 VISIT_START_YMD 변환 완료
- 컬럼 VISIT_END_YMD 변환 완료
- 컬럼 ROAD_NM_ADDR 변환 완료
- 컬럼 LOTNO_ADDR 변환 완료
- 컬럼 X_COORD 변환 완료
- 컬럼 Y_COORD 변환 완료
- 컬럼 ROAD_NM_CD 변환 완료
- 컬럼 LOTNO_CD 변환 완료
- 컬럼 POI_ID 변환 완료
- 컬럼 POI_NM 변환 완료
- 컬럼 RESIDENCE_TIME_MIN 변환 완료
- 컬럼 VISIT_AREA_TYPE_CD 변환 완료
- 컬럼 REVISIT_YN 변환 완료
- 컬럼 VISIT_CHC_REASON_CD 변환 완료
- 컬럼 LODGING_TYPE_CD 변환 완료
- 컬럼 DGSTFN 변환 완료
- 컬럼 REVISIT_INTENTION 변환 완료
- 컬럼 RCMDTN_INTENTION 변환 완료
- 컬럼 SGG_CD 변환 완료
- 컬럼 REGION 변환 완료


In [32]:
change_datatype_of_column(total_dfs_23)

특정 파일 처리: tn_visit_area_info_방문지정보
- 컬럼 VISIT_AREA_ID 변환 완료
- 컬럼 TRAVEL_ID 변환 완료
- 컬럼 VISIT_ORDER 변환 완료
- 컬럼 VISIT_AREA_NM 변환 완료
- 컬럼 VISIT_START_YMD 변환 완료
- 컬럼 VISIT_END_YMD 변환 완료
- 컬럼 ROAD_NM_ADDR 변환 완료
- 컬럼 LOTNO_ADDR 변환 완료
- 컬럼 X_COORD 변환 완료
- 컬럼 Y_COORD 변환 완료
- 컬럼 ROAD_NM_CD 변환 완료
- 컬럼 LOTNO_CD 변환 완료
- 컬럼 POI_ID 변환 완료
- 컬럼 POI_NM 변환 완료
- 컬럼 RESIDENCE_TIME_MIN 변환 완료
- 컬럼 VISIT_AREA_TYPE_CD 변환 완료
- 컬럼 REVISIT_YN 변환 완료
- 컬럼 VISIT_CHC_REASON_CD 변환 완료
- 컬럼 LODGING_TYPE_CD 변환 완료
- 컬럼 DGSTFN 변환 완료
- 컬럼 REVISIT_INTENTION 변환 완료
- 컬럼 RCMDTN_INTENTION 변환 완료
- 컬럼 SGG_CD 변환 완료
- 컬럼 REGION 변환 완료


In [37]:
# total_dfs_23["tn_visit_area_info_방문지정보"].to_parquet("data/visit_23.parquet", engine="pyarrow")

In [40]:
# 지역별 데이터 통합, 저장 함수 생성
def store_total_region_combine(data_dict=None, dir1="aihub", dir2="2022", is_gps="no", file_type="csv"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2) 
    
    if data_dict:
        if file_type == "csv":
                    # gps 데이터가 아니라면
            if is_gps == "no":
                combine_path = os.path.join(base_path, "total_combined", "region_data")
            else:
                combine_path = os.path.join(base_path, "total_combined", "gps_data")

            os.makedirs(combine_path, exist_ok=True)

            for key in data_dict.keys():
                print(f"{key} 데이터셋 저장 시작 \n저장 경로: {combine_path}\n")
                data_dict[key].to_csv(os.path.join(combine_path, f"{key}.{file_type}"), index=False)
            
            print(f"========================== {dir2} 통합 데이터 {len(data_dict.keys())}개 저장 완료 ==========================")
            print(f"=================== {combine_path} 폴더 내 데이터 갯수: {len(os.listdir(combine_path))} ==================== ")

    else:
        combine_path = os.path.join(base_path, "total_combined", "gps_data_parquet")
        os.makedirs(combine_path, exist_ok=True)
        # 디렉토리 내 모든 parquet 파일 목록 가져오기
        
        parquet_files = [os.path.join(combine_path, f) for f in os.listdir(combine_path) if f.endswith(".parquet")]
        dataframes = []

        for file in tqdm(parquet_files):

            df = pd.read_parquet(file, engine="pyarrow")
            dataframes.append(df)


        combined_df = pd.concat(dataframes, ignore_index=True)
        
        # 결과를 Parquet 파일로 저장
        output_file = os.path.join(combine_path, f"total_gps_{dir2}.parquet")
        combined_df.to_parquet(output_file, index=False)
        print(f"========================== {combine_path} 통합 데이터 {combined_df.shape} 저장 완료 ==========================")

In [41]:
store_total_region_combine(total_dfs_22, dir1="aihub", dir2="2022", is_gps="no", file_type="csv")

tc_codea_코드A 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tc_codeb_코드B 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tc_sgg_시군구코드 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_activity_his_활동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_companion_info_동반자정보 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_dat

In [42]:
store_total_region_combine(total_dfs_23, dir1="aihub", dir2="2023", is_gps="no", file_type="csv")

tc_codea_코드A 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tc_codeb_코드B 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tc_sgg_시군구코드 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_activity_his_활동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_companion_info_동반자정보 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_dat

In [4]:
store_total_region_combine(dir1="aihub", dir2="2022", is_gps="yes", file_type="parquet")

100%|██████████| 4/4 [00:14<00:00,  3.50s/it]


========================== data\aihub\2022\total_combined\gps_data_parquet 통합 데이터 (63765731, 7) 저장 완료 ==========================


In [5]:
store_total_region_combine(dir1="aihub", dir2="2023", is_gps="yes", file_type="parquet")


100%|██████████| 6/6 [00:20<00:00,  3.40s/it]


========================== data\aihub\2023\total_combined\gps_data_parquet 통합 데이터 (110895757, 7) 저장 완료 ==========================


### 5-2. 메타데이터 파일 복사
- `data\aihub\metadata` 경로로 파일 복사

In [83]:
# 메타데이터 파일 모두 같다
# 메타데이터 파일 하나만 대표로 이동
metadata = "aihub\\2023\\145.국내 여행로그 데이터_수도권_2차년도\\3.개방데이터\\1.데이터\\Other\\Other\\메타데이터\\"
source_path = os.path.join("data", metadata)
# print(source_path)

# # 대상 폴더 경로
# meta_path = os.path.join("data", "aihub", "metadata")
# os.makedirs(meta_path, exist_ok=True)

# # 파일 복사
# shutil.copy(source_path, meta_path)

# print(f"메타데이터 파일이 {meta_path}로 성공적으로 복사되었습니다!")

In [85]:
meta_df = pd.read_csv(os.path.join(source_path, "tn_poi_master_POIMaster_all.csv"))

In [93]:
meta_df.isnull().sum()

POI_ID                 0
POI_NM                 5
BRNO                   0
SGG_CD                 0
ROAD_NM_ADDR     1072481
LOTNO_ADDR             0
ASORT_LCLASDC    6508130
ASORT_MLSFCDC    6508130
ASORT_SDASDC     6508130
X_COORD                0
Y_COORD                0
ROAD_NM_CD             0
LOTNO_CD               0
dtype: int64

In [95]:
meta_df['SGG_CD'] = meta_df['SGG_CD'].astype(str)
meta_df['LOTNO_CD'] = meta_df['LOTNO_CD'].astype(str)

meta_parquet_path = os.path.join(source_path, "tn_poi_master_POIMaster_all.parquet")
meta_df.to_parquet(meta_parquet_path, engine="pyarrow", index=False)

print(f"Saved Parquet {meta_parquet_path}")

Saved Parquet data\aihub\2023\145.국내 여행로그 데이터_수도권_2차년도\3.개방데이터\1.데이터\Other\Other\메타데이터\tn_poi_master_POIMaster_all.parquet


In [96]:
# 대상 폴더 경로
meta_path = os.path.join("data", "aihub", "metadata")
os.makedirs(meta_path, exist_ok=True)

# 파일 복사
shutil.copy(os.path.join(source_path, "tn_poi_master_POIMaster_all.parquet"), meta_path)

print(f"메타데이터 파일이 {meta_path}로 성공적으로 복사되었습니다!")

메타데이터 파일이 data\aihub\metadata로 성공적으로 복사되었습니다!


### 5-3. 통합한 gps 파일과 원본 gps 파일 TRAVEL_ID 비교

In [19]:
combine_path_22 = os.path.join("data", "aihub", "2022", "total_combined")
combine_path_23 = os.path.join("data", "aihub", "2023", "total_combined")

In [21]:
# parquet 파일 2개 불러온다
gps_22 = pd.read_parquet(os.path.join(combine_path_22, "gps_data_parquet", "total_gps_2022.parquet"), engine="pyarrow")
gps_23 = pd.read_parquet(os.path.join(combine_path_23, "gps_data_parquet", "total_gps_2023.parquet"), engine="pyarrow")

In [23]:
print(f"2022년 통합 gps 데이터 shape:  {gps_22.shape}")
print(f"2023년 통합 gps 데이터 shape: {gps_23.shape}")

2022년 통합 gps 데이터 shape:  (63765731, 7)
2023년 통합 gps 데이터 shape: (110895757, 7)


In [46]:
def compare_gps_each_and_parquet(df, year):
    each_gps_path = os.path.join("data", "aihub", f"{year}", "total_combined", "gps_data")

    unique_id = df["TRAVEL_ID"].unique()

    each_gps = os.listdir(each_gps_path)

    # 정규표현식을 사용하여 다양한 패턴 추출
    if year == 2022:
        pattern = r'(d_d\d+|c_c\d+|b_b\d+|a_a\d+)'  # 원하는 패턴 정의
    else:
        pattern = r'(h_h\d+|g_g\d+|f_f\d+|e_e\d+)'
    extracted = [re.search(pattern, item).group() for item in each_gps if re.search(pattern, item)]


    # 배열을 리스트로 변환 (넘파이 배열은 집합 연산을 지원하지 않음)
    set_each_gps = set(extracted)
    set_gps_parquet = set(unique_id.tolist())  # 배열을 리스트로 변환 후 집합으로 변환

    # 차집합 구하기
    difference = set_each_gps - set_gps_parquet  # each_gps에서 unique_id에 없는 값
    print(f"{year} 기준 개별적인 gps 파일로만 존재하는 TRAVEL_ID: {len(difference)}개 (각각의 gps 파일: {len(set_each_gps)}, 통합된 gps parquet 파일: {len(set_gps_parquet)})\n")
    print(f"{difference}")

In [47]:
compare_gps_each_and_parquet(gps_22, 2022)

2022 기준 개별적인 gps 파일로만 존재하는 TRAVEL_ID: 14개 (각각의 gps 파일: 14400, 통합된 gps parquet 파일: 14386)

{'b_b015954', 'b_b011737', 'a_a007374', 'b_b013792', 'c_c011206', 'b_b018597', 'c_c005232', 'c_c010932', 'b_b011232', 'b_b010148', 'a_a017876', 'a_a017795', 'b_b018574', 'b_b013756'}


In [48]:
compare_gps_each_and_parquet(gps_23, 2023)

2023 기준 개별적인 gps 파일로만 존재하는 TRAVEL_ID: 0개 (각각의 gps 파일: 11520, 통합된 gps parquet 파일: 11520)

set()


In [ ]:
# # 대상 폴더와 찾을 패턴 정의

# 파일 크기를 읽able한 형식으로 변환하는 함수
def format_file_size(size_in_bytes):
    if size_in_bytes < 1024:
        return f"{size_in_bytes} Bytes"
    elif size_in_bytes < 1024**2:
        return f"{size_in_bytes / 1024:.2f} KB"
    elif size_in_bytes < 1024**3:
        return f"{size_in_bytes / 1024**2:.2f} MB"
    else:
        return f"{size_in_bytes / 1024**3:.2f} GB"

# 파일 검색 및 크기 포함 출력
def find_files_in_subfolders_with_scandir(year, search_keywords):
    if year == 2022:
        folder_path = os.path.join("data", "aihub", "2022")
    else: 
        folder_path = os.path.join("data", "aihub", "2023")
    found_files = []
    for root, dirs, files in os.walk(folder_path):
        with os.scandir(root) as entries:
            for entry in entries:
                if entry.is_file() and any(keyword in entry.name for keyword in search_keywords):
                    file_path = os.path.join(root, entry.name)
                    file_size = os.path.getsize(file_path)  # 파일 크기 가져오기
                    found_files.append((file_path, file_size))  # 파일 경로와 크기 추가
    return found_files

In [ ]:
# 결과 확인
# compare_gps_each_and_parquet(gps_22, 2022) 함수 돌렸을 떄 나오는 TRAVEL_ID 가져온다

search_keywords = {'b_b015954', 'b_b011737', 'a_a007374', 'b_b013792', 'c_c011206', 
                   'b_b018597', 'c_c005232', 'c_c010932', 'b_b011232', 'b_b010148', 
                   'a_a017876', 'a_a017795', 'b_b018574', 'b_b013756'}

matching_files = find_files_in_subfolders_with_scandir(2022, search_keywords)
for file_path, file_size in matching_files:
    print("")
    print(f"File: {file_path}")
    print(f"Size: {format_file_size(file_size)}")

# search_keywords에 있는 파일: 48 Bytes → HEADER만 존재하는 파일들이라서 concat 한 데이터프레임에는 해당 ID 없다


File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\01.원천데이터\TS_gps_data\tn_gps_coord_a_a007374.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\01.원천데이터\TS_gps_data\tn_gps_coord_a_a017795.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\01.원천데이터\TS_gps_data\tn_gps_coord_a_a017876.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\01.원천데이터\TS_gps_data\tn_gps_coord_c_c005232.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\02.라벨링데이터\TL_gps_data\tn_gps_coord_a_a007374.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\02.라벨링데이터\TL_gps_data\tn_gps_coord_a_a017795.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\02.라벨링데이터\TL_gps_data\tn_gps_coord_a_a017876.csv
Size: 48 Bytes

File: data\aihub\2022\277.국내 여행로그 데이터(수도권)\01-1.정식개방데이터\Training\02.라벨링데이터\TL_gps_data\tn_gps